## Парсинг "Аналитика процентных ставок в банках" с сайта Финуслуги
##### link https://finuslugi.ru/vklady/analitika_stavok_po_vkladam

Для начала поймем, как именно нумеруются банки на самом сайте

In [23]:
import pandas as pd
import json
import requests
import pandas as pd
import re

In [2]:
root_url = "https://finuslugi.ru/money_data/Root.json"
deposits_url = "https://finuslugi.ru/money_data/DepositsMonitoring.json"

## In order to not get blocked
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept": "*/*",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8,ru;q=0.7",
    "Connection": "keep-alive"
}

# Send GET request
response = requests.get(root_url, headers=headers)

if response.status_code == 200:
    root_data = response.json()
    bank_dict = {bank['id']: bank['name'] for bank in root_data['companies']}
    print('succes')
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")

succes


Мы знаем информацию о названии и нумерации большей части банков
Перейдем к парсингу информации о ставках

In [4]:
response = requests.get(deposits_url, headers=headers)

if response.status_code == 200:
    data = response.json()
    rates_current = data.get("rates", {})
    rates_week_ago = data.get("rates_week_ago", {})
    current_rates_df = pd.DataFrame(rates_current).drop(columns=['date'])
    previous_rates_df = pd.DataFrame(rates_week_ago).drop(columns=['date'])
    current_rates_df.columns = ['current_' + col for col in current_rates_df.columns]
    previous_rates_df.columns = ['previous_' + col for col in previous_rates_df.columns]
    rates_df = pd.concat([current_rates_df, previous_rates_df], axis=1)
    deposits_data = data['deposits']
    dep_df = pd.json_normalize(deposits_data)
    print('succes')
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")




succes


In [5]:
dep_df['bank_name'] = dep_df['bank_id'].map(bank_dict)
dep_df = dep_df.sort_values(by="rate_3", ascending=False).reset_index(drop=True)
dep_df[[
    # "id",
        "bank_id",
        "bank_name",
        "rate_3",
        "rate_6",
        "rate_12", 
        "rate_18",
        "rate_24", 
        "rate_36",
        "is_platform",
        "is_commercial",
        "is_cpa",
        # "link.link_3", 
        # "link.link_6",
        # "link.link_12", 
        # "link.link_18",
        # "link.link_24",
        # "link.link_36"
]].head(31)


,bank_id,bank_name,rate_3,rate_6,rate_12,rate_18,rate_24,rate_36,is_platform,is_commercial,is_cpa
0,23,Московский Кредитный Банк,23.00,23.50,22.00,20.00,20.20,21.00,1,0,0
1,7,Абсолют Банк,21.50,22.00,18.50,11.00,10.40,11.25,0,0,0
2,13,Банк ДОМ.РФ,21.30,22.40,22.90,13.80,13.70,14.25,1,0,0
3,4,Газпромбанк,21.20,22.20,21.20,19.10,20.00,20.00,0,0,0
4,45,Т-Банк,21.00,20.00,20.00,20.00,19.78,0.00,0,0,0
5,89,Металлинвестбанк,21.00,21.50,22.50,22.00,22.00,23.00,0,0,0
6,87,Экспобанк,20.50,22.10,21.55,0.00,0.00,18.85,0,0,0
7,33,Росбанк,20.33,20.84,21.93,0.00,0.00,0.00,0,0,0
8,143,Энерготрансбанк,20.30,20.40,0.00,0.00,0.00,0.00,1,0,0
9,49,Хоум Банк,20.00,22.00,22.00,18.50,17.50,16.50,0,0,0


#### скрипт готов- проверяем, информация по каким банкам впринципе доступна, и какая есть информация по ключевым банкам.

In [6]:
target_banks = [
        "Банк ДОМ.РФ", "Московский Кредитный Банк", "Альфа-Банк", 
        "ВТБ","СберБанк", "Газпромбанк", "Совкомбанк",
        "Россельхозбанк", "Росбанк", "Т-Банк", "Почта Банк","МТС Банк",
        "Ренессанс Кредит",'Транскапиталбанк',"Абсолют Банк",
        "Банк Русский Стандарт", "Яндекс Банк", "Ресо Кредит"
    ]
unique_banks = dep_df['bank_name'].unique()

In [7]:
available_banks = [bank for bank in target_banks if bank in unique_banks]
missing_banks = [bank for bank in target_banks if bank not in unique_banks]
print("\nAvailable banks in the data:")
print(available_banks)
print("\nMissing banks (not found in the data):")
print(missing_banks)


Available banks in the data:
['Банк ДОМ.РФ', 'Московский Кредитный Банк', 'Альфа-Банк', 'СберБанк', 'Газпромбанк', 'Совкомбанк', 'Россельхозбанк', 'Росбанк', 'Т-Банк', 'Почта Банк', 'МТС Банк', 'Абсолют Банк', 'Яндекс Банк']

Missing banks (not found in the data):
['ВТБ', 'Ренессанс Кредит', 'Транскапиталбанк', 'Банк Русский Стандарт', 'Ресо Кредит']


In [8]:
dep_df_filtered = dep_df[dep_df['bank_name'].isin(available_banks)].reset_index(drop=True)
dep_df_filtered['bank_name'] = pd.Categorical(dep_df_filtered['bank_name'], categories=target_banks, ordered=True)
dep_df_filtered = dep_df_filtered.sort_values(by='bank_name').reset_index(drop=True)

dep_df_filtered[[
        # "id",
        "bank_id",
        "bank_name",
        "rate_3",
        "rate_6",
        "rate_12", 
        "rate_18",
        "rate_24", 
        "rate_36",
        "is_platform",
        "is_commercial",
        "is_cpa",
        # "link.link_3", 
        # "link.link_6",
        # "link.link_12", 
        # "link.link_18",
        # "link.link_24",
        # "link.link_36"
]].head(18)

,bank_id,bank_name,rate_3,rate_6,rate_12,rate_18,rate_24,rate_36,is_platform,is_commercial,is_cpa
0,13,Банк ДОМ.РФ,21.30,22.40,22.90,13.80,13.70,14.25,1,0,0
1,23,Московский Кредитный Банк,23.00,23.50,22.00,20.00,20.20,21.00,1,0,0
2,3,Альфа-Банк,18.71,21.05,19.22,17.23,16.21,16.40,0,0,0
3,1,СберБанк,18.50,18.50,18.00,14.50,14.50,12.00,0,1,0
4,4,Газпромбанк,21.20,22.20,21.20,19.10,20.00,20.00,0,0,0
5,43,Совкомбанк,19.10,20.60,20.60,0.00,20.60,21.00,0,0,0
6,35,Россельхозбанк,20.00,23.00,19.00,8.20,8.60,15.00,0,0,0
7,33,Росбанк,20.33,20.84,21.93,0.00,0.00,0.00,0,0,0
8,45,Т-Банк,21.00,20.00,20.00,20.00,19.78,0.00,0,0,0
9,60,Почта Банк,19.50,21.00,18.00,10.00,0.00,10.00,0,0,0


In [10]:
average_rates = dep_df_filtered[[
    "rate_3",
    "rate_6",
    "rate_12",
    "rate_18",
    "rate_24",
    "rate_36"
]].apply(lambda x: x[x != 0].mean())
average_rates

rate_3     19.928333
rate_6     19.886154
rate_12    19.026923
rate_18    14.870000
rate_24    15.709000
rate_36    15.655556
dtype: float64

In [11]:
rates_df

,current_platform,current_top_50,current_top_10,previous_platform,previous_top_50,previous_top_10
rate_3,19.216667,19.102667,19.938750,18.983334,18.391136,18.391136
rate_6,21.120769,20.508409,21.306250,20.965,19.723183,19.723183
rate_12,21.108000,19.513636,19.877500,21.084545,18.68841,18.68841
rate_18,16.610000,15.719310,15.718571,16.258333,15.5227585,15.5227585
rate_24,16.214286,15.914706,17.527143,16.025,15.642941,15.642941
rate_36,16.091667,15.757500,17.057143,16.8125,15.564375,15.564375


## Парсер с Финуслуг информацию обо всех вкладах

На самом сайте при попытке открыть https://finuslugi.ru/vklady/
Мы вызываем через их API json-файл со всеми вкладами, что известны маркетплейсу
Таким образом- мы можем вызывать и смотреть информацию по всем известным им вкладам конкретного банка

In [91]:
# URL API с данными о вкладах
url = "https://finuslugi.ru/deposits/api/proxy/money_data/Deposits.json"
head = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept": "*/*",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8,ru;q=0.7",
    "Connection": "keep-alive"
}

In [92]:
def fetch_deposit_data(url, headers=None):
    """
    Выполняет GET-запрос к указанному URL и возвращает данные в формате JSON.

    :param url: str, URL API с данными о вкладах.
    :param headers: dict, заголовки для запроса (по умолчанию None).
    :return: dict, данные в формате JSON.
    """
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при выполнении запроса: {e}")
        return None


def process_bank_deposits(data, bank_id):
    """
    Обрабатывает данные о вкладах для указанного банка.

    :param data: dict, данные в формате JSON.
    :param bank_id: int, идентификатор банка.
    :return: pd.DataFrame, обработанные данные о вкладах.
    """
    if not data or 'deposits' not in data:
        print("Invalid data format.")
        return pd.DataFrame()
    
    deposits = [d for d in data['deposits'] if d['bank_id'] == bank_id]
    processed_data = []
    
    for deposit in deposits:
        base_info = {
            'bank_id': deposit['bank_id'],
            'bank_name': deposit['bank_name'],
            'deposit_id': deposit['id'],
            'deposit_name': deposit['title'],
            'deposit_url': deposit.get('online_application', {}).get('url'),
            'currency_id': deposit['currency_id'],
            'refill': deposit.get('refill', {}).get('text', 'Not specified'),
            'partial_withdrawal': deposit.get('partial_withdrawal', {}).get('text', 'Not specified'),
            'interest_payment': deposit.get('interest_payment', {}).get('text', 'Not specified'),
            'is_savings_account': 1 if 'накопительный' in deposit['title'].lower() else 0
        }

        # Process rates for each term and amount combination
        for rate in deposit['interest_rate']['rates']:
            term_from = rate.get('term_from')
            amount_from = rate.get('amount_from')
            interest_rate = rate.get('rate')
            
            if term_from is not None and amount_from is not None and interest_rate is not None:
                rate_info = {
                    'term_days': term_from,
                    'min_amount': amount_from,
                    'interest_rate': interest_rate
                }
                combined_info = {**base_info, **rate_info}
                processed_data.append(combined_info)

    return pd.DataFrame(processed_data)

def create_pivot_table(processed_data):
    """
    Создает сводную таблицу на основе обработанных данных о вкладах.
    Возвращает DataFrame с процентными ставками по срокам и суммам.
    """
    if processed_data.empty:
        return processed_data
    
    # Determine unique term_days across all deposits for consistency
    unique_terms = sorted(processed_data['term_days'].unique())
    
    # Create pivot table
    pivot_df = processed_data.pivot_table(
        index=['bank_id', 'bank_name', 'deposit_id', 'deposit_name', 'deposit_url', 'currency_id', 'refill', 'partial_withdrawal', 'interest_payment', 'is_savings_account', 'min_amount'],
        columns='term_days',
        values='interest_rate',
        aggfunc='first'
    ).reset_index()

    # Rename columns to 'rate_{term}_days' format
    pivot_df.columns = [f'rate_{col}_days' if isinstance(col, int) else col for col in pivot_df.columns]
    
    # Add any missing term columns as None
    for term in unique_terms:
        col_name = f'rate_{term}_days'
        if col_name not in pivot_df.columns:
            pivot_df[col_name] = None

    # Sort columns by term_days in ascending order
    fixed_columns = [col for col in pivot_df.columns if not col.startswith('rate_')]
    rate_columns = sorted([col for col in pivot_df.columns if col.startswith('rate_')], key=lambda x: int(x.split('_')[1]))
    pivot_df = pivot_df[fixed_columns + rate_columns]
    
    return pivot_df

def get_sorted_deposits(url, bank_id, headers=None):
    """
    Получает и обрабатывает данные о вкладах для указанного банка, возвращая отсортированный DataFrame.

    :param url: str, URL API с данными о вкладах.
    :param bank_id: int, идентификатор банка.
    :param headers: dict, заголовки для запроса (по умолчанию None).
    :return: pd.DataFrame, отсортированные данные о вкладах.
    """
    data = fetch_deposit_data(url, headers)
    df = process_bank_deposits(data, bank_id)
    if df.empty:
        print("No data available.")
        return df

    return create_pivot_table(df)


In [93]:
bank_id = 154  # Bank ID for VTB, as example
vtb_deposits_df = get_sorted_deposits(url, bank_id, headers=headers)
vtb_deposits_df

,bank_id,bank_name,deposit_id,deposit_name,deposit_url,currency_id,refill,partial_withdrawal,interest_payment,is_savings_account,min_amount,rate_1_days,rate_91_days,rate_92_days,rate_181_days,rate_370_days,rate_548_days,rate_730_days,rate_1095_days
0,154,ВТБ,889,Накопительный ВТБ-Счет,https://www.vtb.ru/personal/vklady-i-scheta/vt...,1,Возможно,Возможно,Ежемесячно,1,1000,22.0,NaN,12.0,NaN,NaN,NaN,NaN,NaN
1,154,ВТБ,889,Накопительный ВТБ-Счет,https://www.vtb.ru/personal/vklady-i-scheta/vt...,1,Возможно,Возможно,Ежемесячно,1,1000000,12.0,NaN,12.0,NaN,NaN,NaN,NaN,NaN
2,154,ВТБ,10005,Новое время,https://www.vtb.ru/personal/vklady-i-scheta/no...,1,Не предусмотрено,Невозможно,Ежемесячно,0,30000,NaN,NaN,NaN,15.27,NaN,NaN,NaN,NaN
3,154,ВТБ,10748,Выгодное начало,https://www.vtb.ru/personal/vklady-i-scheta/vi...,1,Не предусмотрено,Невозможно,Ежемесячно,0,30000,NaN,NaN,NaN,17.37,NaN,NaN,NaN,NaN
4,154,ВТБ,10823,Накопительный счет «Сейф»,https://www.vtb.ru/personal/vklady-i-scheta/na...,1,Возможно,Возможно,Ежемесячно,1,1000,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,154,ВТБ,15148,ВТБ-Вклад «Ключевой» с пополнением,https://www.vtb.ru/privilegia/premialnye-produ...,1,Возможно,Невозможно,Ежемесячно,0,5000000,NaN,NaN,NaN,18.70,18.9,19.0,19.1,19.2
67,154,ВТБ,15148,ВТБ-Вклад «Ключевой» с пополнением,https://www.vtb.ru/privilegia/premialnye-produ...,1,Возможно,Невозможно,Ежемесячно,0,15000000,NaN,NaN,NaN,19.20,19.4,19.5,19.6,19.7
68,154,ВТБ,15149,ВТБ-Вклад «Ключевой» с пополнением и снятием,https://www.vtb.ru/privilegia/premialnye-produ...,1,Возможно,Возможно,Ежемесячно,0,1000000,NaN,NaN,NaN,18.00,18.2,18.3,18.4,18.5
69,154,ВТБ,15149,ВТБ-Вклад «Ключевой» с пополнением и снятием,https://www.vtb.ru/privilegia/premialnye-produ...,1,Возможно,Возможно,Ежемесячно,0,5000000,NaN,NaN,NaN,18.50,18.7,18.8,18.9,19.0


In [94]:
mkb_bank_id= 23
mkb_deposits_df = get_sorted_deposits(url, mkb_bank_id,headers=head)

# Отображение первых строк DataFrame
mkb_deposits_df

,bank_id,bank_name,deposit_id,deposit_name,deposit_url,currency_id,refill,partial_withdrawal,interest_payment,is_savings_account,...,rate_1_days,rate_31_days,rate_60_days,rate_95_days,rate_185_days,rate_271_days,rate_370_days,rate_540_days,rate_730_days,rate_1100_days
0,23,Московский Кредитный Банк,405,Все включено Максимальный доход (USD),https://mkb.ru/personal/deposits/allinclusive/,2,Не предусмотрено,Невозможно,В конце срока,0,...,NaN,NaN,NaN,0.50,0.50,NaN,0.60,0.50,0.50,0.50
1,23,Московский Кредитный Банк,406,Все включено Максимальный доход (EUR),https://mkb.ru/personal/deposits/allinclusive/,3,Не предусмотрено,Невозможно,В конце срока,0,...,NaN,NaN,NaN,0.01,0.01,NaN,0.01,0.01,0.01,0.01
2,23,Московский Кредитный Банк,408,Все включено Накопительный (USD),https://mkb.ru/personal/deposits/allinclusive/,2,Возможно,Невозможно,Ежемесячно,1,...,NaN,NaN,NaN,0.25,0.25,NaN,0.25,0.25,0.25,0.25
3,23,Московский Кредитный Банк,409,Все включено Накопительный (EUR),https://mkb.ru/personal/deposits/allinclusive/,3,Возможно,Невозможно,Ежемесячно,1,...,NaN,NaN,NaN,0.01,0.01,NaN,0.01,0.01,0.01,0.01
4,23,Московский Кредитный Банк,411,Все включено Расчетный (USD),https://mkb.ru/personal/deposits/allinclusive/,2,Возможно,Возможно,Ежемесячно,0,...,NaN,NaN,NaN,0.25,0.25,NaN,0.25,0.25,0.20,0.20
5,23,Московский Кредитный Банк,412,Все включено Расчетный (EUR),https://mkb.ru/personal/deposits/allinclusive/,3,Возможно,Возможно,Ежемесячно,0,...,NaN,NaN,NaN,0.01,0.01,NaN,0.01,0.01,0.01,0.01
6,23,Московский Кредитный Банк,7233,Накопительный счет,https://mkb.ru/personal/deposits/savings-account,1,Возможно,Возможно,Ежемесячно,1,...,7.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,23,Московский Кредитный Банк,10400,МКБ. Преимущество,https://lk.finuslugi.ru/add-product?aggregator...,1,Не предусмотрено,Невозможно,В конце срока,0,...,NaN,17.0,17.0,21.50,22.00,NaN,22.00,20.00,20.00,21.00
8,23,Московский Кредитный Банк,13739,Накопительный счет (USD),https://mkb.ru/personal/deposits/savings-account,2,Возможно,Возможно,Ежемесячно,1,...,0.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,23,Московский Кредитный Банк,13740,Накопительный счет (EUR),https://mkb.ru/personal/deposits/savings-account,3,Возможно,Возможно,Ежемесячно,1,...,0.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
def split_by_currency(df):
    """
    Splits the DataFrame into separate DataFrames based on the currency_id.

    :param df: pd.DataFrame, the original DataFrame with deposits.
    :return: dict of DataFrames split by currency (RUB, USD, EUR, CNY).
    """
    currency_map = {
        1: 'RUB',  # Russian Ruble
        2: 'USD',  # US Dollar
        3: 'EUR',  # Euro
        4: 'CNY'   # Chinese Yuan
    }
    
    # Initialize an empty dictionary to hold the DataFrames for each currency
    currency_dfs = {}
    
    for currency_id, currency_name in currency_map.items():
        # Filter the original DataFrame for each currency and store it in the dictionary
        currency_dfs[currency_name] = df[df['currency_id'] == currency_id].sort_values(by=['deposit_id', 'min_amount']).reset_index(drop=True)
    
    return currency_dfs

def split_by_savings_account(df):
    """
    Splits the DataFrame into two DataFrames based on whether the deposit is a savings account.

    :param df: pd.DataFrame, the original DataFrame with deposits.
    :return: tuple of DataFrames (savings_accounts_df, non_savings_accounts_df).
    """
    # Filter DataFrame for savings accounts and non-savings accounts
    savings_accounts_df = df[df['is_savings_account'] == 1].sort_values(by=['deposit_id', 'min_amount']).reset_index(drop=True)
    non_savings_accounts_df = df[df['is_savings_account'] == 0].sort_values(by=['deposit_id', 'min_amount']).reset_index(drop=True)
    
    return savings_accounts_df, non_savings_accounts_df


In [105]:
# Split by currency
currency_dfs = split_by_currency(vtb_deposits_df)
rub_df = currency_dfs['RUB']  # Deposits in RUB
usd_df = currency_dfs['USD']  # Deposits in USD
eur_df = currency_dfs['EUR']  # Deposits in EUR
cny_df = currency_dfs['CNY']  # Deposits in CNY

# Split by savings account status
savings_accounts_df, non_savings_accounts_df = split_by_savings_account(vtb_deposits_df)


In [106]:
savings_accounts_df

,bank_id,bank_name,deposit_id,deposit_name,deposit_url,currency_id,refill,partial_withdrawal,interest_payment,is_savings_account,min_amount,rate_1_days,rate_91_days,rate_92_days,rate_181_days,rate_370_days,rate_548_days,rate_730_days,rate_1095_days
0,154,ВТБ,889,Накопительный ВТБ-Счет,https://www.vtb.ru/personal/vklady-i-scheta/vt...,1,Возможно,Возможно,Ежемесячно,1,1000,22.0,NaN,12.0,NaN,NaN,NaN,NaN,NaN
1,154,ВТБ,889,Накопительный ВТБ-Счет,https://www.vtb.ru/personal/vklady-i-scheta/vt...,1,Возможно,Возможно,Ежемесячно,1,1000000,12.0,NaN,12.0,NaN,NaN,NaN,NaN,NaN
2,154,ВТБ,10823,Накопительный счет «Сейф»,https://www.vtb.ru/personal/vklady-i-scheta/na...,1,Возможно,Возможно,Ежемесячно,1,1000,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [113]:
non_savings_accounts_df.tail(6)

,bank_id,bank_name,deposit_id,deposit_name,deposit_url,currency_id,refill,partial_withdrawal,interest_payment,is_savings_account,min_amount,rate_1_days,rate_91_days,rate_92_days,rate_181_days,rate_370_days,rate_548_days,rate_730_days,rate_1095_days
62,154,ВТБ,15148,ВТБ-Вклад «Ключевой» с пополнением,https://www.vtb.ru/privilegia/premialnye-produ...,1,Возможно,Невозможно,Ежемесячно,0,1000000,NaN,NaN,NaN,18.2,18.4,18.5,18.6,18.7
63,154,ВТБ,15148,ВТБ-Вклад «Ключевой» с пополнением,https://www.vtb.ru/privilegia/premialnye-produ...,1,Возможно,Невозможно,Ежемесячно,0,5000000,NaN,NaN,NaN,18.7,18.9,19.0,19.1,19.2
64,154,ВТБ,15148,ВТБ-Вклад «Ключевой» с пополнением,https://www.vtb.ru/privilegia/premialnye-produ...,1,Возможно,Невозможно,Ежемесячно,0,15000000,NaN,NaN,NaN,19.2,19.4,19.5,19.6,19.7
65,154,ВТБ,15149,ВТБ-Вклад «Ключевой» с пополнением и снятием,https://www.vtb.ru/privilegia/premialnye-produ...,1,Возможно,Возможно,Ежемесячно,0,1000000,NaN,NaN,NaN,18.0,18.2,18.3,18.4,18.5
66,154,ВТБ,15149,ВТБ-Вклад «Ключевой» с пополнением и снятием,https://www.vtb.ru/privilegia/premialnye-produ...,1,Возможно,Возможно,Ежемесячно,0,5000000,NaN,NaN,NaN,18.5,18.7,18.8,18.9,19.0
67,154,ВТБ,15149,ВТБ-Вклад «Ключевой» с пополнением и снятием,https://www.vtb.ru/privilegia/premialnye-produ...,1,Возможно,Возможно,Ежемесячно,0,15000000,NaN,NaN,NaN,19.0,19.2,19.3,19.4,19.5


In [103]:
dom_rf_bank_id= 13
dom_rf_deposits_df = get_sorted_deposits(url, dom_rf_bank_id,headers=head)

# Отображение первых строк DataFrame
dom_rf_deposits_df

,bank_id,bank_name,deposit_id,deposit_name,deposit_url,currency_id,refill,partial_withdrawal,interest_payment,is_savings_account,min_amount,rate_1_days,rate_91_days,rate_92_days,rate_181_days,rate_367_days,rate_550_days,rate_730_days,rate_1100_days
0,13,Банк ДОМ.РФ,4615,Накопительный счет,https://domrfbank.ru/deposits/savings-account/,1,Возможно,Возможно,Ежемесячно,1,1,19.5,NaN,6.0,NaN,NaN,NaN,NaN,NaN
1,13,Банк ДОМ.РФ,13633,Накопительный счет (CNY),https://domrfbank.ru/deposits/savings-account/,4,Возможно,Возможно,Ежемесячно,1,1,0.7,NaN,0.4,NaN,NaN,NaN,NaN,NaN
2,13,Банк ДОМ.РФ,14070,Мой дом с пополнением и снятием (CNY),https://domrfbank.ru/deposits/vklad-moy-dom/,4,Возможно,Возможно,Ежемесячно / В конце срока,0,500,NaN,4.80,NaN,4.90,5.10,4.40,3.90,3.40
3,13,Банк ДОМ.РФ,14071,Мой дом с пополнением (CNY),https://domrfbank.ru/deposits/vklad-moy-dom/,4,Возможно,Невозможно,Ежемесячно / В конце срока,0,500,NaN,4.90,NaN,5.00,5.20,4.60,4.10,3.60
4,13,Банк ДОМ.РФ,14072,Мой дом (CNY),https://domrfbank.ru/deposits/vklad-moy-dom/,4,Не предусмотрено,Невозможно,Ежемесячно / В конце срока,0,500,NaN,6.30,NaN,6.30,6.30,6.20,6.10,6.00
5,13,Банк ДОМ.РФ,14306,Надежный Премиум,https://lk.finuslugi.ru/add-product?aggregator...,1,Не предусмотрено,Невозможно,В конце срока,0,3000000,NaN,15.40,NaN,15.65,15.80,13.80,13.70,14.25
6,13,Банк ДОМ.РФ,14312,Надежный VIP,https://lk.finuslugi.ru/add-product?aggregator...,1,Не предусмотрено,Невозможно,В конце срока,0,15000000,NaN,15.40,NaN,15.65,15.80,13.80,13.70,14.25
7,13,Банк ДОМ.РФ,14323,Надежный,https://lk.finuslugi.ru/add-product?aggregator...,1,Не предусмотрено,Невозможно,В конце срока,0,10000,NaN,21.10,NaN,22.20,22.70,14.90,14.80,14.80
8,13,Банк ДОМ.РФ,14323,Надежный,https://lk.finuslugi.ru/add-product?aggregator...,1,Не предусмотрено,Невозможно,В конце срока,0,1000000,NaN,21.30,NaN,22.40,22.90,15.10,15.00,15.00
9,13,Банк ДОМ.РФ,14371,Мой дом (USD),https://domrfbank.ru/deposits/vklad-moy-dom/,2,Не предусмотрено,Невозможно,Ежемесячно / В конце срока,0,500,NaN,3.00,NaN,3.20,3.70,0.01,0.01,0.01


In [85]:
def fetch_deposit_data(url, headers=None):
    """
    Выполняет GET-запрос к указанному URL и возвращает данные в формате JSON.

    :param url: str, URL API с данными о вкладах.
    :param headers: dict, заголовки для запроса (по умолчанию None).
    :return: dict, данные в формате JSON.
    """
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при выполнении запроса: {e}")
        return None

def process_bank_deposits(data, bank_id):
    """
    Обрабатывает данные о вкладах для указанного банка.

    :param data: dict, данные в формате JSON.
    :param bank_id: int, идентификатор банка.
    :return: pd.DataFrame, обработанные данные о вкладах.
    """
    if not data or 'deposits' not in data:
        print("Некорректные данные.")
        return pd.DataFrame()

    # Фильтрация вкладов по bank_id
    bank_deposits = [deposit for deposit in data['deposits'] if deposit['bank_id'] == bank_id]

    processed_data = []
    for deposit in bank_deposits:
        # Основная информация о вкладе
        deposit_info = {
            'bank_id': deposit['bank_id'],
            'bank_name': deposit['bank_name'],
            'deposit_id': deposit['id'],
            'deposit_name': deposit['title'],
            'deposit_url': deposit.get('online_application', {}).get('url'),
            'currency_id': deposit['currency_id'],
            'min_amount': deposit['amount'].get('from'),
            'max_amount': deposit['amount'].get('to'),
            'min_term_days': deposit['term'].get('from'),
            'max_term_days': deposit['term'].get('to'),
            'refill': deposit.get('refill', {}).get('text', 'Не указано'),
            'partial_withdrawal': deposit.get('partial_withdrawal', {}).get('text', 'Не указано'),
            'interest_payment': deposit.get('interest_payment', {}).get('text', 'Не указано'),
            'is_savings_account': 1 if 'накопительный' in deposit['title'].lower() else 0
        }
        for rate in deposit['interest_rate']['rates']:
            term_from = rate.get('term_from')
            amount_from = rate.get('amount_from')
            interest_rate = rate.get('rate')
            if term_from is not None and amount_from is not None and interest_rate is not None:
                rate_info = {
                    'term_days': term_from,
                    'min_amount': amount_from,
                    'interest_rate': interest_rate
                    }
                combined_info = {**base_info, **rate_info}
                processed_data.append(combined_info)

    return pd.DataFrame(processed_data)

# def sort_rate_columns(df):
#     """
#     Сортирует столбцы ставок по вкладам по количеству дней и сумме, указанным в их названиях.

#     :param df: pd.DataFrame, исходный DataFrame.
#     :return: pd.DataFrame, DataFrame с отсортированными столбцами.
#     """
#     # Функция для извлечения числовых значений дней и суммы из названия столбца
#     def extract_days_amount(column_name):
#         match = re.search(r'rate_(\d+)_days_(\d+)_amount', column_name)
#         if match:
#             days = int(match.group(1))
#             amount = int(match.group(2))
#             return (days, amount)
#         return (float('inf'), float('inf'))

#     # Отбор столбцов, содержащих информацию о ставках
#     rate_columns = [col for col in df.columns if 'rate_' in col]

#     # Сортировка столбцов по количеству дней и сумме
#     sorted_rate_columns = sorted(rate_columns, key=extract_days_amount)

#     # Определение нового порядка столбцов
#     new_column_order = [col for col in df.columns if col not in rate_columns] + sorted_rate_columns

#     # Переупорядочивание DataFrame
#     sorted_df = df[new_column_order]

#     return sorted_df

def create_pivot_table(processed_data):
    """
    Создает сводную таблицу на основе обработанных данных о вкладах.
    Возвращает DataFrame с процентными ставками по срокам и суммам.
    """
    df = pd.DataFrame(processed_data)

    # Определяем все уникальные значения term_days
    unique_terms = sorted(df['term_days'].unique())

    # Создаем сводную таблицу
    pivot_df = df.pivot_table(
        index=['bank_id', 'bank_name', 'deposit_id', 'deposit_name', 'deposit_url', 'currency_id', 'refill', 'partial_withdrawal', 'interest_payment', 'is_savings_account', 'min_amount'],
        columns='term_days',
        values='interest_rate',
        aggfunc='first'
    ).reset_index()

    # Переименовываем колонки
    pivot_df.columns.name = None
    pivot_df = pivot_df.rename(columns=lambda x: f'rate_{x}_days' if isinstance(x, int) else x)

    # Добавляем отсутствующие сроки с None
    for term in unique_terms:
        col_name = f'rate_{term}_days'
        if col_name not in pivot_df.columns:
            pivot_df[col_name] = None

    # Упорядочиваем колонки
    fixed_columns = ['bank_id', 'bank_name', 'deposit_id', 'deposit_name', 'deposit_url', 'currency_id', 'refill', 'partial_withdrawal', 'interest_payment', 'is_savings_account', 'min_amount']
    rate_columns = [f'rate_{term}_days' for term in unique_terms]
    pivot_df = pivot_df[fixed_columns + rate_columns]

    return pivot_df


def get_sorted_deposits(url, bank_id, headers=None):
    """
    Получает и обрабатывает данные о вкладах для указанного банка, возвращая отсортированный DataFrame.

    :param url: str, URL API с данными о вкладах.
    :param bank_id: int, идентификатор банка.
    :param headers: dict, заголовки для запроса (по умолчанию None).
    :return: pd.DataFrame, отсортированные данные о вкладах.
    """
    data = fetch_deposit_data(url, headers)
    df = process_bank_deposits(data, bank_id)
    if df.empty:
        print("Нет данных для отображения.")
        return df

    # Сортировка столбцов ставок
    sorted_df = create_pivot_table(df)
    return sorted_df

# # Пример использования

# def fetch_deposit_data(url, headers=head):
#     """
#     Выполняет GET-запрос к указанному URL и возвращает данные в формате JSON.

#     :param url: str, URL API с данными о вкладах.
#     :param headers: dict, заголовки для запроса (по умолчанию None).
#     :return: dict, данные в формате JSON.
#     """
#     try:
#         response = requests.get(url, headers=headers)
#         response.raise_for_status()
#         return response.json()
#     except requests.exceptions.RequestException as e:
#         print(f"Ошибка при выполнении запроса: {e}")
#         return None

# def process_bank_deposits(data, bank_id):
#     """
#     Обрабатывает данные о вкладах для указанного банка.

#     :param data: dict, данные в формате JSON.
#     :param bank_id: int, идентификатор банка.
#     :return: pd.DataFrame, обработанные данные о вкладах.
#     """
#     if not data or 'deposits' not in data:
#         print("Некорректные данные.")
#         return pd.DataFrame()

#     # Фильтрация вкладов по bank_id
#     bank_deposits = [deposit for deposit in data['deposits'] if deposit['bank_id'] == bank_id]

#     processed_data = []
    # for deposit in bank_deposits:
    #     # Основная информация о вкладе
    #     deposit_info = {
    #         'bank_id': deposit['bank_id'],
    #         'bank_name': deposit['bank_name'],
    #         'deposit_id': deposit['id'],
    #         'deposit_name': deposit['title'],
    #         'deposit_url': deposit.get('online_application', {}).get('url'),
    #         'currency_id': deposit['currency_id'],
    #         'min_amount': deposit['amount'].get('from'),
    #         'refill': deposit.get('refill', {}).get('text', 'Не указано'),
    #         'partial_withdrawal': deposit.get('partial_withdrawal', {}).get('text', 'Не указано'),
    #         'interest_payment': deposit.get('interest_payment', {}).get('text', 'Не указано'),
    #         'is_savings_account': 1 if 'накопительный' in deposit['title'].lower() else 0
    #     }

    #     # Выбор ставок для минимальной суммы вклада, сортировка по сроку
    #     min_amount = int(deposit_info['min_amount']) if deposit_info['min_amount'] else None
    #     rates = sorted(
    #         [rate for rate in deposit['interest_rate']['rates'] if rate.get('amount_from') == min_amount],
    #         key=lambda x: x['term_from'] if x['term_from'] is not None else float('inf')
    #     )

    #     # Добавление ставок по срокам в информацию о вкладе
    #     for rate in rates:
    #         term_days = rate['term_from']
    #         if term_days is not None:
#                 deposit_info[f'rate_{term_days}_days'] = rate['rate']

#         processed_data.append(deposit_info)

#     return pd.DataFrame(processed_data)

# def sort_rate_columns_by_days(df):
#     """
#     Сортирует столбцы ставок по вкладам по количеству дней, указанному в их названиях.

#     Параметры:
#     df (pd.DataFrame): Исходный DataFrame.

#     Возвращает:
#     pd.DataFrame: DataFrame с отсортированными столбцами.
#     """
#     # Функция для извлечения числового значения дней из названия столбца
#     def extract_days(column_name):
#         match = re.search(r'rate_(\d+)_days', column_name)
#         return int(match.group(1)) if match else float('inf')

#     # Отбор столбцов, содержащих информацию о ставках
#     rate_columns = [col for col in df.columns if 'rate_' in col]

#     # Сортировка столбцов по количеству дней
#     sorted_rate_columns = sorted(rate_columns, key=extract_days)

#     # Определение нового порядка столбцов
#     new_column_order = [col for col in df.columns if col not in rate_columns] + sorted_rate_columns

#     # Переупорядочивание DataFrame
#     sorted_df = df[new_column_order]

#     return sorted_df



# def get_sorted_deposits(url, bank_id, headers=head):
#     """
    # Получает и обрабатывает данные о вкладах для указанного банка, возвращая отсортированный DataFrame.

    # :param url: str, URL API с данными о вкладах.
    # :param bank_id: int, идентификатор банка.
    # :param headers: dict, заголовки для запроса (по умолчанию None).
    # :return: pd.DataFrame, отсортированные данные о вкладах.
    # """
    # data = fetch_deposit_data(url, headers)
    # df = process_bank_deposits(data, bank_id)
    # if df.empty:
    #     print("Нет данных для отображения.")
    #     return df

    # # Сортировка DataFrame по 'deposit_name'
    # sorted_df = sort_rate_columns_by_days(df)
    # return sorted_df
